# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


# -----

In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-01-21 01:28:47,732 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
clickstream_df = se.read.csv('hdfs:/data/clickstream.csv',  header=True, sep="\t",  inferSchema=True)
clickstream_df.createOrReplaceTempView("clickstream")

## Spark SQL Solution

In [3]:
query = """
    WITH base AS (
        SELECT *
        FROM clickstream
        ORDER BY user_id, session_id, timestamp
    )

    , filtered_sessions AS (
        SELECT 
            *
            ,SUM(CASE WHEN event_type LIKE '%error%' THEN 1 ELSE 0 END) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) as error_count
        FROM base
        WHERE 1=1
    )

    ,valid_sessions AS (
        SELECT
            user_id
            ,session_id
            ,event_page
            ,timestamp
        FROM filtered_sessions
        WHERE 1=1
            AND error_count = 0
            AND event_type = 'page'
    )

    ,user_routes AS (
        SELECT 
            user_id
            ,session_id
            ,CONCAT_WS('-', COLLECT_LIST(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp, event_page ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)) as route
        FROM valid_sessions
    )

    SELECT
        route
        ,COUNT(*) as count
    FROM (SELECT DISTINCT * FROM user_routes)
    GROUP BY route
    ORDER BY count DESC
    LIMIT 30
"""

top_routes_sql = se.sql(query)
top_routes_sql.show(30, truncate=False)

+---------------------+-----+
|route                |count|
+---------------------+-----+
|main                 |8090 |
|main-archive         |1095 |
|main-rabota          |1039 |
|main-internet        |879  |
|main-bonus           |865  |
|main-news            |760  |
|main-tariffs         |669  |
|main-online          |584  |
|main-vklad           |514  |
|main-archive-rabota  |167  |
|main-rabota-archive  |167  |
|main-bonus-archive   |139  |
|main-rabota-bonus    |136  |
|main-bonus-rabota    |134  |
|main-news-rabota     |134  |
|main-archive-internet|131  |
|main-rabota-news     |129  |
|main-internet-rabota |128  |
|main-archive-news    |125  |
|main-rabota-internet |123  |
|main-internet-archive|123  |
|main-archive-bonus   |117  |
|main-tariffs-internet|114  |
|main-internet-bonus  |114  |
|main-news-archive    |112  |
|main-news-internet   |108  |
|main-archive-tariffs |103  |
|main-internet-news   |102  |
|main-tariffs-archive |102  |
|main-main            |94   |
+---------

## Spark RDD solution

In [4]:
# converting df to RDD
clickstream_rdd = clickstream_df.rdd.persist()

# finding error sessions
error_sessions = (clickstream_rdd
                  .filter(lambda row: 'error' in row.event_type.lower())
                  .map(lambda row: ((row.user_id, row.session_id), row.timestamp))
                  .reduceByKey(lambda x, y: min(x, y)))

# keeping only valid sessions
valid_sessions = (clickstream_rdd
                  .filter(lambda row: row.event_type.lower() == 'page')
                  .map(lambda row: ((row.user_id, row.session_id), (row.event_page, row.timestamp)))
                  .leftOuterJoin(error_sessions)
                  .filter(lambda x: x[1][1] is None or (x[1][0][1] < x[1][1]))
                  .map(lambda x: (x[0], x[1][0]))  # Keep only user_id, session_id, event_page, timestamp
                  .filter(lambda x: x[1][1] is not None))  # Ensure timestamp is not None

user_routes = (valid_sessions.groupByKey().mapValues(lambda x: '-'.join([page for page, timestamp in sorted(x, key=lambda y: (y[1], y[0]))])))

route_counts = (user_routes.map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x + y))

top_routes_rdd = route_counts.takeOrdered(30, key=lambda x: -x[1])

for route, count in top_routes_rdd:
    # just formatting things
    print(route, (23 - len(route)) * ' ', count)

main                     8090
main-archive             1095
main-rabota              1039
main-internet            879
main-bonus               865
main-news                760
main-tariffs             669
main-online              584
main-vklad               514
main-archive-rabota      167
main-rabota-archive      167
main-bonus-archive       139
main-rabota-bonus        136
main-news-rabota         134
main-bonus-rabota        134
main-archive-internet    131
main-rabota-news         129
main-internet-rabota     128
main-archive-news        125
main-internet-archive    123
main-rabota-internet     123
main-archive-bonus       117
main-tariffs-internet    114
main-internet-bonus      114
main-news-archive        112
main-news-internet       108
main-archive-tariffs     103
main-internet-news       102
main-tariffs-archive     102
main-main                94


## Spark DataFrame Solution

In [5]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

ordered_clickstream_df = clickstream_df.orderBy("user_id", "session_id", "event_page", "timestamp")

# finding error events and filtering them out
windowSpec = Window.partitionBy("user_id", "session_id").orderBy("timestamp")

filtered_sessions_df = ordered_clickstream_df.withColumn("error_count", F.sum(F.when(F.col("event_type").like("%error%"), 1).otherwise(0)).over(windowSpec))

valid_sessions_df = filtered_sessions_df.filter((F.col("error_count") == 0) & (F.col("event_type") == "page"))

user_routes_df = valid_sessions_df.withColumn("route", F.concat_ws('-', F.collect_list("event_page").over(windowSpec.rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)))).select("user_id", "session_id", "route")

distinct_routes_df = user_routes_df.distinct()

top_routes_df = distinct_routes_df.groupBy("route").count().orderBy(F.col("count").desc()).limit(30)

top_routes_df.show(30, truncate=False)

+---------------------+-----+
|route                |count|
+---------------------+-----+
|main                 |8090 |
|main-archive         |1095 |
|main-rabota          |1039 |
|main-internet        |879  |
|main-bonus           |865  |
|main-news            |760  |
|main-tariffs         |669  |
|main-online          |584  |
|main-vklad           |514  |
|main-archive-rabota  |167  |
|main-rabota-archive  |167  |
|main-bonus-archive   |139  |
|main-rabota-bonus    |136  |
|main-bonus-rabota    |134  |
|main-news-rabota     |134  |
|main-archive-internet|131  |
|main-rabota-news     |129  |
|main-internet-rabota |128  |
|main-archive-news    |125  |
|main-rabota-internet |123  |
|main-internet-archive|123  |
|main-archive-bonus   |117  |
|main-internet-bonus  |114  |
|main-tariffs-internet|114  |
|main-news-archive    |112  |
|main-news-internet   |108  |
|main-archive-tariffs |103  |
|main-internet-news   |102  |
|main-tariffs-archive |102  |
|main-main            |94   |
+---------

## Saving outputs

In [6]:
# SQL
pandas_df_sql = top_routes_sql.toPandas()
pandas_df_sql.to_csv('sql_solution.csv', sep='\t', index=False)

# RDD
spark_df_rdd = se.createDataFrame(top_routes_rdd, schema=['route', 'count'])
pandas_df_rdd = spark_df_rdd.toPandas()
pandas_df_rdd.to_csv('rdd_solution.csv', sep='\t', index=False)

# DataFrame
pandas_df_df = top_routes_df.toPandas()
pandas_df_df.to_csv('df_solution.csv', sep='\t', index=False)

In [7]:
sc.stop()